In [1]:
import os

from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

os.environ["AZURE_OPENAI_API_KEY"] = "38002279fa574fbcba0fbc424ae056d5"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://lumin-azure-openai-rnd.openai.azure.com/"

model = AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    azure_deployment='lumin-rnd-gpt-4-32k',
)

In [4]:
from langchain import LLMChain
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
long_prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)


llm_chain = LLMChain(
    prompt=long_prompt,
    llm=model
)

qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]

res = llm_chain.generate(qs)
res

LLMResult(generations=[[ChatGeneration(text='The Green Bay Packers won the Super Bowl in the 2010 season.', generation_info={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, message=AIMessage(content='The Green Bay Packers won the Super Bowl in the 2010 season.', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}))], [ChatGeneration(text='You are 193 centimeters tall.', generation_info={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, message=AIMessage(content='You are 193 centimeters tall.', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}))], [ChatGeneration(text='Alan Shepard', generation_info={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, message=AIMessage(content='Alan Shepard', response_metadata={'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}))], [ChatGeneration(text='A blade of grass does not h